In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
data=pd.read_csv("../data/datos_modificados.csv")
data=data.drop(columns="id")

In [3]:
data_usable=data[data["x_e_out [-]"].notnull()].copy()
prediction=data[data["x_e_out [-]"].isnull()].copy()

In [7]:
X = data_usable.drop('x_e_out [-]', axis=1)
y = data_usable['x_e_out [-]']

# Identificar las columnas categóricas
categorical_cols = ['author', 'geometry']

# Definir el transformador para la codificación one-hot
ct = ColumnTransformer([('one_hot_encoder', OneHotEncoder(), categorical_cols)], remainder='passthrough')

# Aplicar la codificación one-hot a las variables categóricas
X_encoded = ct.fit_transform(X)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42, shuffle=True)

In [9]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [17]:
svm = SVR()

# Define los parámetros que deseas probar en la búsqueda en cuadrícula
parameters = {
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'C': [0.1, 1, 10],
    'epsilon': [0.1, 0.2, 0.3],
    'gamma': ['scale', 'auto']
}

# Realiza la búsqueda en cuadrícula
grid_search = GridSearchCV(estimator=svm, param_grid=parameters, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train_scaled, y_train)

# Obtiene los mejores parámetros y el mejor modelo
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Realiza predicciones en el conjunto de prueba con el mejor modelo
y_pred = best_model.predict(X_test_scaled)

In [18]:
# Calcula el RMSE en el conjunto de prueba
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE:", rmse)

RMSE: 0.07965810821588803


In [20]:
print(best_params)

{'C': 1, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}


In [11]:
y_pred = svm.predict(X_test_scaled)

In [15]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

RMSE: 0.07965810821588803
